<a href="https://colab.research.google.com/github/raj-aws/learnpython/blob/main/pymain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q groq wikipedia-api elevenlabs pydub
!apt-get install -y ffmpeg

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [2]:
import wikipediaapi
from groq import Groq
from elevenlabs.client import ElevenLabs
from pydub import AudioSegment
from google.colab import userdata
import io
import os
import ssl

# --- QUICK FIX FOR SSL ERROR ---
ssl._create_default_https_context = ssl._create_unverified_context

# --- 1. CONFIGURATION ---
# Replace these strings with your actual keys
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
ELEVEN_API_KEY = userdata.get('ELEVEN_API_KEY')


# Voice IDs (George for Male, Rachel for Female)
VOICE_RAJ = "hczKB0VbXLcBTn17ShYS"
VOICE_PALLAVI = "QbPozfpTbDrP62pzQpyg"

# Initialize Clients
client_groq = Groq(api_key=GROQ_API_KEY)
client_eleven = ElevenLabs(api_key=ELEVEN_API_KEY)

# --- 2. DATA INGESTION ---
def fetch_wiki_data(topic):
    print(f"Fetching Wikipedia data for: {topic}...")
    # Added a more descriptive user_agent as required by Wiki
    wiki = wikipediaapi.Wikipedia(user_agent="MyPodcastProject/1.0 (contact:rajmca2002@gmail.com)", language='en')
    page = wiki.page(topic)
    if not page.exists():
        return "Topic not found."
    return page.summary[:3000]

# --- 3. INTELLIGENCE (Script Generation) ---
def generate_hinglish_script(context):
    print("Generating hinglish script via Groq...")
    prompt = f"""
    Context Data: {context}
    Task: Write a natural 2-minute Hinglish conversation between RAJ and PALLAVI.
    Style: Use 'Mumbai' Hinglish. Mix Hindi and English naturally. Keep 50% English and 50% Telugu.
    Format:
    Raj: [dialogue]
    Pallavi: [dialogue]
    """

    response = client_groq.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.3-70b-versatile",
    )
    return response.choices[0].message.content



def generate_telugu_script(context):
    print("Generating Telugu-English script...")
    chat_completion = client_groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a professional Telugu scriptwriter.
                Write a conversation between Raj and Pallavi about the topic provided.
                Rules:
                1. Use 'Tenglish' (A natural mix of Telugu and English).
                2. Use casual Telugu words like 'emundi', 'mariyu', 'nijama', 'avunu'.
                3. Keep technical cricket terms in English (e.g., 'Captaincy', 'Playoffs').
                4. Format as Raj: [text] and Pallavi: [text]."""
            },
            {"role": "user", "content": f"Topic data: {context}"}
        ],
        model="llama-3.3-70b-versatile",
    )
    return chat_completion.choices[0].message.content


def run_audio_from_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        saved_script = f.read()
    create_multivoice_audio(saved_script)

# run_audio_from_file("podcast_script.txt")


# --- 4. SYNTHESIS & ASSEMBLY ---
def create_multivoice_audio(script_text):
    print("Starting audio synthesis...")
    combined_audio = AudioSegment.empty()
    lines = script_text.strip().split('\n')

    for line in lines:
        if ":" not in line: continue

        speaker, text = line.split(":", 1)
        speaker = speaker.strip()
        text = text.strip()

        # Select voice based on speaker name
        current_voice = VOICE_RAJ if speaker == "Raj" else VOICE_PALLAVI

        print(f"Synthesizing {speaker}'s line...")
        # ElevenLabs SDK v1.x syntax
        audio_stream = client_eleven.text_to_speech.convert(
            voice_id=current_voice,
            model_id="eleven_multilingual_v2",
            text=text
        )

        # Convert stream to audio segment
        audio_data = b"".join(audio_stream)
        segment = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")

        # Add 500ms pause for natural flow
        combined_audio += segment + AudioSegment.silent(duration=500)

    output_path = "local_podcast_output3.mp3"
    combined_audio.export(output_path, format="mp3")
    print(f"✅ Success! File saved at: {os.path.abspath(output_path)}")

# --- 5. EXECUTION ---
if __name__ == "__main__":
    try:
        wiki_text = fetch_wiki_data("Mumbai Indians")
        script = generate_hinglish_script(wiki_text)
        #script = generate_telugu_script(wiki_text)
       # --- NEW: SAVE SCRIPT TO TEXT FILE ---
        with open("podcast_script.txt", "w", encoding="utf-8") as f:
            f.write(script)
        print("✅ Script saved to podcast_script.txt")

        # 3. Create audio (This might still fail if credits are low)
        create_multivoice_audio(script)

    except Exception as e:
        print(f"An error occurred: {e}")



/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Fetching Wikipedia data for: Mumbai Indians...
Generating hinglish script via Groq...
An error occurred: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}
